In [1]:
#Lauren Sisson-Dales, 0751195

In [157]:
%matplotlib inline

In [158]:
import findspark

In [159]:
findspark.init()

In [160]:
findspark.find()

'C:\\Users\\laure\\anaconda3\\lib\\site-packages\\pyspark'

In [161]:
from pyspark.sql import SparkSession

In [162]:
spark = SparkSession.builder.appName("cruise").getOrCreate() #creating spark session

In [163]:
from pyspark.ml.regression import LinearRegression

In [164]:
df = spark.read.csv("C://Users//laure//.ipynb_checkpoints//Cruise_ship_data.csv",inferSchema=True,header=True)

In [165]:
df.printSchema() #looking at data types

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [166]:
df.show() #checking data

+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26| 47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|  110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 70.367|     20.56|  8.55| 10.22|            34.23| 9.2|
|Fascination|   Carnival| 19| 70.367|     20.52|  8.55

In [167]:
df.describe().show() #summary of data

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     null|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [168]:
df.groupBy('Cruise_Line').count().show() #Organizing and counting number of cruiselines

+-----------------+-----+
|      Cruise_Line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [169]:
from pyspark.ml.feature import StringIndexer #package to convert strings

In [170]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="categories")

In [171]:
indexed = indexer.fit(df).transform(df) #fitting to df and transforming

In [172]:
indexed.head(1) #checking transformation, where we have added categorie numbers for cruise lines

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.277, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, categories=16.0)]

In [173]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [174]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'categories']

In [175]:
assembler=VectorAssembler(inputCols=[ 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'categories'],outputCol='features') #adding columns to create a vector of needed categories

In [176]:
output=assembler.transform(indexed) #adding our indexed column from earlier

In [177]:
output.select('features','crew').show() #taking a look at comparison

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.277,6.94,5.94...|3.55|
|[30.277,6.94,5.94...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.239,37.0,9.5...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [178]:
final_data=output.select(['features','crew'])

In [179]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [180]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              103|
|   mean|7.425048543689321|
| stddev|3.273136911697775|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [181]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               55|
|   mean|8.485454545454548|
| stddev|3.834534506580282|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [182]:
from pyspark.ml.regression import LinearRegression

In [183]:
lr=LinearRegression(labelCol='crew')

In [184]:
model=lr.fit(train_data)

In [185]:
results=model.evaluate(test_data)

In [186]:
results.predictions.show()

+--------------------+----+-------------------+
|            features|crew|         prediction|
+--------------------+----+-------------------+
|[2.329,0.94,2.96,...| 0.6| 0.1393727110470957|
|[3.341,0.66,2.79,...|0.59|0.11934042810903067|
|[16.8,2.96,5.14,1...| 2.1| 2.3061598982174645|
|[22.08,8.26,5.78,...| 3.5|  4.788033317402007|
|[25.0,3.82,5.97,1...|2.95| 3.0711546830169927|
|[30.277,6.84,5.94...| 4.0|  4.031331302703991|
|[30.277,6.84,5.94...| 4.0|  4.031331302703991|
|[35.143,12.5,6.69...|5.35|  4.961154613943112|
|[47.262,14.86,7.2...| 6.7|  6.291765101903591|
|[50.76,17.48,7.54...|6.14|  7.395012816525567|
|[50.76,17.48,7.54...|6.14|  7.395012816525567|
|[51.004,9.4,7.81,...|5.45|  6.226254345762904|
|[52.926,13.02,7.1...|6.17|  6.052532268995641|
|[53.049,13.44,7.2...| 6.0|  6.215218929767746|
|[55.451,12.66,7.1...|5.88|    5.8171335729135|
|[58.6,15.66,8.24,...| 7.0|  7.526052130175811|
|[63.0,14.4,7.77,7...|5.31|  6.661455257110285|
|[68.0,10.8,7.9,5....|6.36| 6.7081745935

In [35]:
results.rootMeanSquaredError #excellent

0.6051079383531068

In [36]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               115|
|   mean| 7.955478260869565|
| stddev|3.3016190189743417|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [37]:
results.r2  #very good

0.9766194938736668

In [38]:
results.meanSquaredError

0.3661556170579473

In [39]:
results.meanAbsoluteError

0.4795115893303429

In [40]:
from pyspark.sql.functions import corr #pearson correlation

In [42]:
df.select(corr('crew','passengers')).show() #crew and passengers are highly correlated hence the accuracy

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [43]:
df.select(corr('crew','cabins')).show() #crew and cabins are highly correlated hence the accuracy

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

